installing packages

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install torch
!pip install transformers
!pip install accelerate
!pip install langchain-core
!pip install langchain-google-genai

video to audio using ffmpeg

In [ ]:
import os

def extract_audio(video_path, output_extension="mp3"):
  """Extracts audio from a video file in Drive and saves it with the specified extension."""
  filename_raw, _ = os.path.splitext(os.path.basename(video_path))
  output_path = os.path.join("/content/drive/MyDrive", filename_raw + "-audio." + output_extension)

  # Use ffmpeg to decode and re-encode audio to ensure correct format
  !ffmpeg -hide_banner -y -i "$video_path" -vn -c:a libmp3lame -b:a 192k "$output_path"

  return f"Audio extracted to: {output_path}"

# Example usage (remains the same)
video_file_path = "/content/drive/MyDrive/small.mp4"
result = extract_audio(video_file_path)
print(result)

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/content/drive/MyDrive/small.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: isommp42
    creation_time   : 2024-01-17T13:52:54.000000Z
  Duration: 00:00:58.37, start: 0.000000, bitrate: 1109 kb/s
  Stream #0:0(und): Video: h264 (Main) (avc1 / 0x31637661), yuv420p(tv, bt709), 720x1280 [SAR 1:1 DAR 9:16], 977 kb/s, 30 fps, 30 tbr, 15360 tbn, 60 tbc (default)
    Metadata:
      creation_time   : 2024-01-17T13:52:54.000000Z
      handler_name    : ISO Media file produced by Google Inc. Created on: 01/17/2024.
      vendor_id       : [0][0][0][0]
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      creation_time   : 2024-01-17T13:52:54.000000Z
      handler_name    : ISO Media file produced by Google Inc. Created on: 01/17/2024.
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:1 -> #0:0 (aac (native) -> mp3 (libmp3lame))
Pr

audio to text conversion using whisper model

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=438,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


def transcribe_and_save(audio_path, output_file_path):
    """Transcribes audio and saves the result to a text file in Drive."""
    result = pipe(audio_path)
    transcript = result["text"]

    # Save the transcript with UTF-8 encoding
    with open(output_file_path, "w", encoding="utf-8") as f:
        f.write(transcript)

    with open(output_file_path, "r", encoding="utf-8") as f:  # Read with UTF-8
        transcript_content = f.read()
        print("Transcript Content:\n", transcript_content)

    return f"Transcript saved to: {output_file_path}"


audio_file_path = "/content/drive/MyDrive/small-audio.mp3"
output_file_path = "/content/drive/MyDrive/transcript.txt"  # Path to save the transcript

result = transcribe_and_save(audio_file_path, output_file_path)
print(result)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Transcript Content:
  $1 chair versus $5,000 chair. How comfortable can you get? First up is the $1 chair, which has a really nice folding mechanism, but it's just a bit smaller than I was expecting. Ow! It did not support my weight. Zero out of ten. For $50 though, you can get something you can actually sit on. Oh, my back feels very straight. But calling this comfortable would be a bit of a stretch. Three out of ten. But then we have the $500 Herman Miller Embody, which is designed specifically with ergonomics in mind. This is the kind of chair that's not just good for your posture, but comfortable too. Seven out of ten. For $1,000 though, on the complete opposite end of the spectrum, this is the squidger. Zero hard frames, zero edges, just hundreds of thousands of tiny foam chips. Nine out of ten. For $5,000 then, this, the Weyron Felicity. It's zero gravity, it's heated, it's got stereo speakers next to your ears, touchscreen controlled for the massage functions, and even its own h

summarization of transcript using bart model

In [ ]:
from transformers import pipeline

def summarize_chunks(text, summarizer, chunk_size=1000, max_length=100, min_length=30):
    """Summarizes text by splitting it into chunks and summarizing each chunk."""
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    return summaries

def summarize_from_file(transcript_path, summary_path):
    """Reads text from a transcript file, summarizes it using chunks, and saves/prints the summary."""
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")  # Initialize summarizer here
    with open(transcript_path, "r", encoding="utf-8") as f:
        transcript = f.read()

    # Summarize in chunks
    summaries = summarize_chunks(transcript, summarizer)  # Pass summarizer to the function
    combined_summary = "\n\n".join(summaries)

    with open(summary_path, "w", encoding="utf-8") as f:
        f.write(combined_summary)

    print("Summary:\n", combined_summary)

# Example usage:
transcript_path = "/content/drive/MyDrive/transcript.txt"
summary_path = "/content/drive/MyDrive/summary.txt"
summarize_from_file(transcript_path, summary_path)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Your max_length is set to 100, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Summary:
 For $50 you can get something you can actually sit on. The Herman Miller Embody is designed specifically with ergonomics in mind. For $5,000 then, this is the Weyron Felicity.

E. massage around your entire face. 10 out of 10. e massage around Your entire Face. 10 of the best e Massages ever.


generate keynotes from summary using gemini model

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# Get API key from userdata (make sure you set it using userdata.set('G_KEY', 'YOUR_API_KEY'))
GOOGLE_API_KEY = userdata.get('G_KEY')

def generate_keynotes_from_chunk(transcript_chunk):
    """Generates keynotes from a single chunk of transcript text."""
    try:
        llm = ChatGoogleGenerativeAI(
            model="gemini-pro",  # Specify Gemini-Pro
            google_api_key=GOOGLE_API_KEY,
            max_output_tokens=100)
    except ValueError as e:
        print(f"Error initializing Gemini model: {e}")
        return None

    message = HumanMessage(
        content=f"Generate keynotes from the following transcript chunk, also don't use any beautifers like bold, italics, underscore, etc. Keep it plain text:\n\n{transcript_chunk}, also only accept the family friendly parts from the transcript and generate keynotes according to it. skip all unwanted and illegal parts for filtering"
    )
    try:
        response = llm.invoke([message])
        keynotes = response.content
        return keynotes
    except Exception as e:
        print(f"Error generating keynotes: {e}")
        return None

def generate_and_print_keynotes(transcript_path):
    """Generates keynotes from a transcript in smaller chunks, saves them to Drive, and prints the content."""
    try:
        with open(transcript_path, "r", encoding="utf-8") as file:
            transcript = file.read()
    except FileNotFoundError as e:
        print(f"Error reading transcript file: {e}")
        return None

    chunk_size = 250  # Smaller chunk size (adjust as needed)
    transcript_chunks = [transcript[i:i+chunk_size] for i in range(0, len(transcript), chunk_size)]

    all_keynotes = []
    for chunk in transcript_chunks:
        keynotes = generate_keynotes_from_chunk(chunk)
        if keynotes:
            all_keynotes.append(keynotes)

    keynotes_path = "/content/drive/MyDrive/keynotes.txt"
    try:
        with open(keynotes_path, "w", encoding="utf-8") as f:
            f.write("\n\n".join(all_keynotes))  # Combine keynotes with separators
    except OSError as e:
        print(f"Error writing keynotes to file: {e}")
        return None

    try:
        with open(keynotes_path, "r", encoding="utf-8") as f:
            keynotes_content = f.read()
            print("Keynotes:\n", keynotes_content)
    except FileNotFoundError as e:
        print(f"Error reading keynotes file: {e}")
        return None

# Example usage:
transcript_path = "/content/drive/MyDrive/transcript.txt"  # Replace with your transcript path
generate_and_print_keynotes(transcript_path)

Keynotes:
 - Comfort and price disparity in chairs
- Not meeting expectations
- Lack of support and durability

**Keynotes:**

- Ergonomic design for back support
- Comfort level rated 3 out of 10
- Herman Miller Embody chair designed for ergonomics

- Posture-friendly and comfortable chair
- Squidger chair with no hard frames or edges

**Keynotes:**

* Zero gravity functionality
* Heated seating
* Stereo speakers integrated with headrests
* Touchscreen massage function controls
* Dedicated headset

**Keynotes:**

* Facial massage techniques
* Benefits of facial massage
* Importance of using family-friendly content
